In [7]:
import pandas as pd
import numpy as np
from torch.distributions.constraints import positive

data = pd.read_csv("sp500_returns_with_tickers.csv", index_col="date")
data.head()
cap_data = pd.read_csv("sp500_market_caps.csv", index_col="date")
cap_data.tail()
cap_data = cap_data * 1000
cap_data.tail()
cap_data_numeric = cap_data.apply(pd.to_numeric, errors='coerce')
cap_data_numeric = cap_data_numeric.fillna(0)
weight_data = cap_data_numeric.div(cap_data_numeric.sum(axis=1), axis=0)
row_sums = weight_data.sum(axis=1)
print(row_sums.head())
weight_data.tail()
# (1) Make sure the columns match and are aligned:
data = data[weight_data.columns]

data_numeric = data.apply(pd.to_numeric, errors='coerce').fillna(0)

# Now compute returns:
sp500_returns = (data_numeric * weight_data).sum(axis=1)

sp500_returns
active_mask = cap_data_numeric > 0

# Number of active stocks per month:
n_active_stocks = active_mask.sum(axis=1)

# Equal weight data:
equal_weight_data = active_mask.div(n_active_stocks, axis=0)

# Equal-weighted SP500 returns:
sp500_ew_returns = (data_numeric * equal_weight_data).sum(axis=1)

# Optional: Check
print(sp500_ew_returns.head())
innovation_returns = sp500_returns - sp500_ew_returns
innovation_weights = weight_data - equal_weight_data

innovation_weights.tail()

date
1960-01-29    1.0
1960-02-29    1.0
1960-03-31    1.0
1960-04-29    1.0
1960-05-31    1.0
dtype: float64
date
1960-01-29   -0.049087
1960-02-29    0.007490
1960-03-31   -0.025784
1960-04-29   -0.015902
1960-05-31    0.025842
dtype: float64


,ACF,ABK,AMT,JAVA,AN,ORCL,MSFT,SDS,AYE,TROW,...,CFN,AVGO,VRSK,DG.2,FTNT,VAL,GNRC,QEP,CBOE,TSLA
date,,,,,,,,,,,,,,,,,,,,,
2024-08-30,0.0,0.0,0.0,0.0,0.0,0.005898,0.060810,0.0,0.0,-0.001510,...,0.0,0.013414,-0.001201,-0.001619,-0.000800,0.0,-0.001797,0.0,-0.001553,0.011865
2024-09-30,0.0,0.0,0.0,0.0,0.0,0.007358,0.061306,0.0,0.0,-0.001505,...,0.0,0.013956,-0.001235,-0.001616,-0.000809,0.0,-0.001795,0.0,-0.001560,0.014616
2024-10-31,0.0,0.0,0.0,0.0,0.0,0.007308,0.058396,0.0,0.0,-0.001500,...,0.0,0.013860,-0.001213,-0.001636,-0.000784,0.0,-0.001789,0.0,-0.001541,0.014042
2024-11-29,0.0,0.0,0.0,0.0,0.0,0.007676,0.057414,0.0,0.0,-0.001469,...,0.0,0.012342,-0.001204,-0.001667,-0.000614,0.0,-0.001777,0.0,-0.001562,0.018917
2024-12-31,0.0,0.0,0.0,0.0,0.0,0.006998,0.058433,0.0,0.0,-0.001504,...,0.0,0.018958,-0.001238,-0.001667,-0.000592,0.0,-0.001810,0.0,-0.001594,0.023006


In [3]:
data

,JAVA,ORCL,MSFT,SDS,AYE,TROW,HON,EMC,BEAM,LLTC,...,CFN,AVGO,VRSK,DG.1,FTNT,VAL,GNRC,QEP,CBOE,TSLA
date,,,,,,,,,,,,,,,,,,,,,
2005-01-31,-0.191095,0.003644,-0.016467,-0.050830,-0.018772,-0.037781,0.016097,-0.119032,0.088106,-0.023736,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-02-28,-0.032110,-0.059550,-0.039574,-0.029007,-0.021200,0.025731,0.061041,-0.033588,-0.031555,0.034976,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-03-31,-0.042654,-0.036294,-0.039348,0.321333,0.091389,-0.028995,-0.020016,-0.026856,-0.004568,-0.019201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-04-29,-0.103960,-0.073718,0.046752,-0.031884,0.182962,-0.070899,-0.038968,0.064935,0.048989,-0.064213,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-05-31,0.052486,0.107266,0.022925,0.039222,-0.010638,0.081385,0.018911,0.071646,0.026602,0.048951,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-30,NaN,0.013195,-0.001100,NaN,NaN,-0.071535,0.020708,NaN,NaN,NaN,...,NaN,0.013319,0.042292,-0.310823,0.321675,NaN,0.005460,NaN,0.122718,-0.077391
2024-09-30,NaN,0.206030,0.031548,NaN,NaN,0.038948,-0.005772,NaN,NaN,NaN,...,NaN,0.062707,-0.016385,0.019284,0.010950,NaN,0.015013,NaN,-0.002580,0.221942
2024-10-31,NaN,-0.012676,-0.055659,NaN,NaN,0.008538,-0.004983,NaN,NaN,NaN,...,NaN,-0.015826,0.025228,-0.046589,0.014313,NaN,0.041981,NaN,0.042466,-0.045025


In [ ]:
innovation_weights

,JAVA,ORCL,MSFT,SDS,AYE,TROW,HON,EMC,BEAM,LLTC,...,CFN,AVGO,VRSK,DG.1,FTNT,VAL,GNRC,QEP,CBOE,TSLA
date,,,,,,,,,,,,,,,,,,,,,
2005-01-31,-0.000648,0.004604,0.024247,-0.001287,-0.001756,-0.001288,0.000841,0.000882,-0.000889,-0.000936,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
2005-02-28,-0.000715,0.003997,0.022682,-0.001320,-0.001766,-0.001283,0.000913,0.000735,-0.000947,-0.000919,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
2005-03-31,-0.000738,0.003886,0.021957,-0.001086,-0.001740,-0.001292,0.000904,0.000719,-0.000928,-0.000921,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
2005-04-29,-0.000846,0.003563,0.023587,-0.001098,-0.001685,-0.001328,0.000848,0.000951,-0.000853,-0.000976,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
2005-05-31,-0.000817,0.004002,0.023404,-0.001082,-0.001642,-0.001292,0.000817,0.001079,-0.000853,-0.000954,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-30,0.000000,0.005898,0.060810,0.000000,0.000000,-0.001510,0.000748,0.000000,0.000000,0.000000,...,0.0,0.013414,-0.001201,-0.001619,-0.000800,0.0,-0.001797,0.0,-0.001553,0.011865
2024-09-30,0.000000,0.007358,0.061306,0.000000,0.000000,-0.001505,0.000675,0.000000,0.000000,0.000000,...,0.0,0.013956,-0.001235,-0.001616,-0.000809,0.0,-0.001795,0.0,-0.001560,0.014616
2024-10-31,0.000000,0.007308,0.058396,0.000000,0.000000,-0.001500,0.000685,0.000000,0.000000,0.000000,...,0.0,0.013860,-0.001213,-0.001636,-0.000784,0.0,-0.001789,0.0,-0.001541,0.014042


In [5]:
data = data[innovation_weights.columns]
innovation_constituent_returns = innovation_weights * data
innovation_constituent_returns

,JAVA,ORCL,MSFT,SDS,AYE,TROW,HON,EMC,BEAM,LLTC,...,CFN,AVGO,VRSK,DG.1,FTNT,VAL,GNRC,QEP,CBOE,TSLA
date,,,,,,,,,,,,,,,,,,,,,
2005-01-31,0.000124,0.000017,-0.000399,0.000065,0.000033,0.000049,0.000014,-0.000105,-0.000078,0.000022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-02-28,0.000023,-0.000238,-0.000898,0.000038,0.000037,-0.000033,0.000056,-0.000025,0.000030,-0.000032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-03-31,0.000031,-0.000141,-0.000864,-0.000349,-0.000159,0.000037,-0.000018,-0.000019,0.000004,0.000018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-04-29,0.000088,-0.000263,0.001103,0.000035,-0.000308,0.000094,-0.000033,0.000062,-0.000042,0.000063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-05-31,-0.000043,0.000429,0.000537,-0.000042,0.000017,-0.000105,0.000015,0.000077,-0.000023,-0.000047,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-30,NaN,0.000078,-0.000067,NaN,NaN,0.000108,0.000015,NaN,NaN,NaN,...,NaN,0.000179,-0.000051,0.000503,-0.000257,NaN,-0.000010,NaN,-0.000191,-0.000918
2024-09-30,NaN,0.001516,0.001934,NaN,NaN,-0.000059,-0.000004,NaN,NaN,NaN,...,NaN,0.000875,0.000020,-0.000031,-0.000009,NaN,-0.000027,NaN,0.000004,0.003244
2024-10-31,NaN,-0.000093,-0.003250,NaN,NaN,-0.000013,-0.000003,NaN,NaN,NaN,...,NaN,-0.000219,-0.000031,0.000076,-0.000011,NaN,-0.000075,NaN,-0.000065,-0.000632


In [6]:
innovation_constituent_returns.cov()

,JAVA,ORCL,MSFT,SDS,AYE,TROW,HON,EMC,BEAM,LLTC,...,CFN,AVGO,VRSK,DG.1,FTNT,VAL,GNRC,QEP,CBOE,TSLA
JAVA,2.970638e-08,-5.133051e-08,-9.234451e-08,2.019440e-09,2.271418e-09,8.234108e-09,-4.528260e-09,-3.914330e-09,8.887306e-09,7.360180e-09,...,-1.478103e-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ORCL,-5.133051e-08,2.676684e-07,4.869846e-07,1.920270e-09,-2.439660e-08,-2.027315e-08,2.680380e-08,3.705429e-08,-3.008191e-08,-2.942372e-08,...,-2.457103e-08,1.959702e-08,-1.151440e-08,-3.811461e-09,-1.513708e-08,-1.370116e-08,-3.280274e-08,-3.962259e-08,-4.107383e-09,5.148711e-07
MSFT,-9.234451e-08,4.869846e-07,4.985200e-06,3.767515e-08,-6.465023e-08,-9.044772e-08,8.848986e-08,6.635814e-08,-6.544913e-08,-4.642045e-08,...,-2.484785e-08,4.620989e-07,-1.038237e-07,-4.290942e-08,-1.369012e-07,-8.255553e-08,-3.537234e-07,-5.287776e-08,-7.738725e-08,5.576362e-06
SDS,2.019440e-09,1.920270e-09,3.767515e-08,1.989921e-08,4.335867e-09,-8.168104e-10,1.745983e-09,-5.502380e-10,-2.003415e-09,-2.164720e-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AYE,2.271418e-09,-2.439660e-08,-6.465023e-08,4.335867e-09,1.456300e-08,3.688420e-09,-4.075569e-09,-3.892200e-09,7.705219e-09,4.023572e-09,...,3.721789e-10,NaN,NaN,NaN,NaN,NaN,NaN,1.971060e-09,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VAL,NaN,-1.370116e-08,-8.255553e-08,NaN,NaN,2.316889e-09,-2.148078e-09,-9.848908e-10,-1.902700e-09,3.217148e-09,...,1.086156e-09,-3.087876e-10,6.656587e-09,-2.013953e-09,NaN,3.663895e-08,NaN,8.342349e-09,NaN,NaN
GNRC,NaN,-3.280274e-08,-3.537234e-07,NaN,NaN,1.701426e-08,-1.228098e-09,NaN,NaN,NaN,...,NaN,2.451045e-08,3.119097e-09,-2.950429e-10,5.620086e-09,NaN,7.563701e-08,NaN,5.535903e-09,-5.257157e-07
QEP,NaN,-3.962259e-08,-5.287776e-08,NaN,1.971060e-09,3.494698e-09,-5.682851e-09,-1.068828e-08,4.990255e-09,4.905922e-09,...,3.392464e-09,2.434027e-09,NaN,4.790335e-10,NaN,8.342349e-09,NaN,2.624586e-08,NaN,NaN
CBOE,NaN,-4.107383e-09,-7.738725e-08,NaN,NaN,2.229503e-09,-4.334161e-09,NaN,NaN,NaN,...,NaN,1.400590e-08,1.933646e-09,-1.082248e-11,3.196282e-09,NaN,5.535903e-09,NaN,9.912342e-09,-1.897471e-08
